**What is Retrieval Augmented Generation (RAG)?**

AI are trained on large sets of information, but that information can quickly become outdated or may not cover the specific details you care about. For example, if you ask an AI about a recent event or something unique to your company, it might not know the answer or could even make up a response that sounds correct but isn’t accurate. This is known as a “hallucination,” and it’s a common challenge with AIs.

RAG helps solve this problem by adding an additoinal step that uses an AI to search for relavant data in real time and use that information to answer questions. Instead of guessing, the AI can pull up the exact details from your files, policies, or records, making its answers more reliable and easier to verify.

To enable RAG, an important step is preparing your data so it is searchable. This usually means storing your documents in a format that allows the AI to quickly retrieve specific passages. In practice, this could involve setting up a specialized database for documents (often a “vector database”) or indexing your existing content in a way that supports efficient lookups. In some cases, it may also involve cleaning or restructuring your data to ensure consistency.

In the following notebook you will take a deep look at the steps required to setup your own RAG system. In this example PDF files will be used to help give more information to the LLM about the attention layer using the "Attention Is All You Need" PDF.

![RAG Flow](rag-architecture-diagram.png)

---
First, install the required dependencies.

In [ ]:
!pip install langchain==0.3.26
!pip install langchain_chroma==0.2.5
!pip install langchain_community==0.3.27
!pip install langchain-together==0.3.1
!pip install pypdf==5.7.0

---
Environment variables will be configured for use throughout the notebook. This approach centralizes important information, making it easier to update settings in the future. For example, you can quickly change the embedding model or update the API key if it expires.

In [ ]:
TOGETHER_API_KEY = "YOUR_KEY"
DOC_PATH = "docs"
CHROMA_PATH = "chroma_vectors"
EMBEDDING_MODEL = "BAAI/bge-base-en-v1.5"

---
Download a PDF for processing

In [ ]:
!mkdir -p docs
!wget -O docs/attention-genai.pdf https://raw.githubusercontent.com/Brian-McGinn/GenAI-Tutorials/main/rag-tutorial/docs/attention-genai.pdf

---
Before running any models, it is important to load the documents. The following function is designed to extract documents from a directory and create a list of Document objects using the LangChain schema. This example includes support for PDF and Markdown files, but it can be extended to accommodate additional file types as needed. For more information, refer to the [LangChain Document Loaders documentation](https://python.langchain.com/docs/integrations/document_loaders/).

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.schema import Document
import os

def load_documents_from_directory(directory: str=DOC_PATH) -> list[Document]:
    """
    Load all PDF and Markdown files from a directory into LangChain Document objects.
    """
    documents = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        
        if filename.lower().endswith(".pdf"):
            loader = PyPDFLoader(filepath)
            documents.extend(loader.load())
        
        elif filename.lower().endswith(".md"):
            loader = TextLoader(filepath, encoding="utf-8")
            documents.extend(loader.load())
    
    return documents


# Execute Documentation Load
docs = load_documents_from_directory(DOC_PATH)
print(docs[0].page_content.splitlines()[:2])

---
With the documents now loaded, they can be split into smaller chunks to provide a more refined context window. While it is possible to manually split the document contents, LangChain and other APIs offer methods to streamline this process. In this example, the RecursiveCharacterTextSplitter method will be used to divide the document content into 500-character chunks. To help avoid breaking up sentences or words in the middle, a chunk overlap can be specified. This means that the next chunk will begin a set number of characters before the end of the previous chunk, thereby capturing any content that may have been cut off.

For more information, see [Recursive Character Text Splitter](https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
    split_text = text_splitter.split_documents(documents)
    return split_text

# split documents loaded in the previous step
split_docs = split_documents(docs)
print(split_docs[0].page_content[:156])

---
As demonstrated in the previous example, the page content is divided into separate chunks.

Additionally, the RecursiveCharacterTextSplitter() function automatically adds metadata to each data chunk.

In [ ]:
print(split_docs[0].metadata)

---
You can add custom metadata to be used later for retrieval.

In [ ]:
split_docs[0].metadata["keywords"] = "RAG-Tutorial"
split_docs[0].metadata["NewKey"] = "Example key"

print(split_docs[0].metadata["keywords"])
print(split_docs[0].metadata["NewKey"])
print(split_docs[0].metadata)

---
Next, generate embeddings for the chunks using TogetherEmbeddings and the BAAI/bge-base-en-v1.5 model.

In [ ]:
from langchain_together import TogetherEmbeddings
from langchain_chroma import Chroma

def save_vectors(documents: list[Document]):
    embedding = TogetherEmbeddings(model=EMBEDDING_MODEL, api_key=TOGETHER_API_KEY)
    Chroma.from_documents(
        documents=documents,
        embedding=embedding,
        persist_directory=CHROMA_PATH,
    )

# Save the previously created chunks into a local Chroma vector database
save_vectors(split_docs)

---
One method for retrieving data is similarity search. This technique uses the user query and embeddings to identify the most relevant document chunks. By specifying the parameter k, you can return the top k most similar chunks found during the search. Using a smaller k value reduces resource usage and latency but may provide less context for the prompt. Adjusting the k value is important for balancing performance and accuracy.

In [ ]:
def get_embeddings(query):
    if query:
        embedding = TogetherEmbeddings(model=EMBEDDING_MODEL, api_key=TOGETHER_API_KEY)
        vector_store = Chroma(
            persist_directory=CHROMA_PATH,
            embedding_function=embedding
        )
        return vector_store.similarity_search(query, k=3)
    return "No Results Found"

# Get the context chunks based on query 
sim_query = "What is a transformation attention layer?"
sim_context = get_embeddings(sim_query)
print(sim_context)

---
Metadata can also be used to refine your search. This is especially beneficial when searching large datasets or when you know which metadata will yield the most relevant response to your query.

In [ ]:
def get_embeddings_from_attention(query):
    if query:
        embedding = TogetherEmbeddings(model=EMBEDDING_MODEL, api_key=TOGETHER_API_KEY)
        vector_store = Chroma(
            persist_directory=CHROMA_PATH,
            embedding_function=embedding
        )
        return vector_store.similarity_search(query, k=3, filter={'source': 'docs/attention-genai.pdf'})
    return "No Results Found"
 
# Get the context chunks based on query and only from the attention-genai.pdf metadata
meta_query = "Explain the Transformer model architecture."
meta_context = get_embeddings_from_attention(meta_query)
print(meta_context)

---
With the RAG functions complete and executed, it is time to create the prompt templates. Notice that the system prompt explicitly instructs the AI to use only the provided context when answering questions. This ensures that the AI focuses its responses on the supplied data chunks rather than general knowledge. By instructing the AI not to answer when it does not know, we reduce the possibility of hallucinations.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_message = (
    "You are a helpful assistant that ONLY answers questions based on the "
    "provided context. If no relevant context is provided, do NOT answer the query."
)
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_message),
    HumanMessagePromptTemplate.from_template(
        "Context:\n{context}\n\nQuestion: {query}"
    )
])



---
With the prompts configured, it is time to set up an LLM and query the AI using the RAG chunks.

In [ ]:
from langchain_together import ChatTogether

llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    api_key=TOGETHER_API_KEY,
    max_tokens=1000,
    timeout=None,
    max_retries=2,
)

def format_context(docs: list[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in docs])

# User the LLM and retrieve context in the previous 
messages = prompt.format_messages(
    context=format_context(meta_context),
    query=meta_query
)
response = llm.invoke(messages)
print(response.content)

---
**Handling Large-Scale Data:**

While the basic RAG approach covered in this tutorial will work for most datasets, it is important to be aware of advanced techniques that can further enhance accuracy.

- ReRanking: By passing your initial similarity search results to a reranking model (e.g., Salesforce/Llama-Rank-V1), the AI can recalculate the relevance scores of the results and provide a more refined set of context passages. This method can also be used to narrow down search results by retrieving a large number of results in the initial search and then selecting a smaller, more relevant subset with the reranking model. The primary drawback is that this approach requires two searches, which may impact user performance.

- GraphRAG: This approach involves creating a knowledge graph of your data that connects each data point to other relevant data points. Leveraging these relationships helps the search return connected information and provides explanations for those connections. Although this method can yield higher accuracy, transforming your data into a knowledge graph can be computationally expensive.
